In [25]:
# https://pieriantraining.com/tensorflow-lstm-example-a-beginners-guide/
# https://www.tensorflow.org/guide/keras/working_with_rnns

In [26]:
import numpy as np
import pandas as pd
import matplotlib as plt
import tensorflow as tf
import keras
from keras.models import load_model
from keras import layers
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import joblib

In [27]:
dataset = pd.read_csv('../../data/fraction_preprocessed_data.csv',encoding='ISO-8859-1')
dataset = dataset.drop(['Unnamed: 0'], axis=1)
dataset = dataset.dropna(subset=['Text'])
# dataset = dataset.sample(frac=0.1, random_state=42)
dataset

,Text,Source,Human
0,ING AsiaPacific Companys Problems Research Pap...,Human,1
1,Crisis Love Inquiry Essay Critical Writing fol...,Human,1
2,Sure sex segregation makes lot sense many spor...,Human,1
3,Christianity Islam Values Essay Christianity f...,Human,1
4,Becca liked swim practiced everyday hours ente...,GLM-130B,0
...,...,...,...
78887,Mass Eoghan Chada 10 brother Ruairi 5 said St ...,OPT-30B,0
78888,Asian Teachers Polish Lesson Perfection Stigle...,Human,1
78889,Move knife slowly avoid slipping accidentally ...,OPT-6.7B,0
78890,Good dreams likely occur person feeling relaxe...,Text-Davinci-003,0


In [28]:
# Initialize the tokenizer
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(dataset['Text'])

In [29]:
# Convert texts to sequences
sequences = tokenizer.texts_to_sequences(dataset['Text'])

# Pad sequences to ensure uniform input length
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')

In [30]:
# unitializing label encoder
label_encoder = LabelEncoder()
label_encoder.fit(dataset['Human'])

# labels to numerical format conversion
encoded_labels = label_encoder.transform(dataset['Human'])

In [31]:
joblib.dump(tokenizer, '../../models/NeuralNetworks/FeedForward/Variables/tokenizer.pkl')
joblib.dump(label_encoder, '../../models/NeuralNetworks/FeedForward/Variables/label_encoder.pkl')
joblib.dump(encoded_labels, '../../models/NeuralNetworks/FeedForward/Variables/encoded_labels.pkl')
joblib.dump(padded_sequences, '../../models/NeuralNetworks/FeedForward/Variables/padded_sequences.pkl')

['../../models/NeuralNetworks/FeedForward/Variables/padded_sequences.pkl']

In [32]:
# FNN with 1 Hidden Layer
fnn = Sequential([
    Dense(128, activation='relu', input_shape=(100,)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

c:\Users\elija\VSCodeLocal\Learning\Projects\PrinciplesOfML\Project\AIvsHumanGeneratationAnalysis\env\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [33]:
fnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [34]:
# training phase
fnn.fit(padded_sequences, encoded_labels, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10


1973/1973 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.5239 - loss: 11.6145 - val_accuracy: 0.5652 - val_loss: 0.7879
Epoch 2/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.5340 - loss: 0.7548 - val_accuracy: 0.5659 - val_loss: 0.7994
Epoch 3/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.5452 - loss: 0.7178 - val_accuracy: 0.5675 - val_loss: 0.6936
Epoch 4/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5563 - loss: 0.6926 - val_accuracy: 0.5749 - val_loss: 0.6709
Epoch 5/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5703 - loss: 0.6792 - val_accuracy: 0.5912 - val_loss: 0.6667
Epoch 6/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5865 - loss: 0.6711 - val_accuracy: 0.5802 - val_loss: 0.7042
Epoch 7/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5899 - loss: 0.6710 - val_accuracy: 0.5853 - val_loss: 0.6850
Epoch 8/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5966 - loss: 0.6644 - val_accura

In [35]:
fnn.save('../../models/NeuralNetworks/FeedForward/fnn_three_hidden_61.keras')